In [3]:
from retirement_engine.simulator import run_simulation, load_market_data

In [8]:
import os

def load_market_data(etf_source):
    if not os.path.exists(etf_source):
        raise FileNotFoundError(f"Market data file not found: {etf_source}")

    df = pd.read_csv(etf_source)
    if "daily_return" not in df.columns:
        raise ValueError("CSV must contain a 'daily_return' column.")

    return df["daily_return"].astype(float).tolist()

In [9]:
withdrawal_rate = 0.05
sp500_weight = 0.60
etf_source = "C:/Users/JohnFriedrich/OneDrive/Scripts/Python/RetirementEngine/data/market.csv"
strategies = ["Fixed", "Guardrails", "Pause After Loss", "Dynamic"]

In [10]:
results = {}

for strategy in strategies:
    balances, withdrawals = run_simulation(
        withdrawal_rate=withdrawal_rate,
        sp500_weight=sp500_weight,
        etf_source=etf_source,
        strategy=strategy
    )
    results[strategy] = {
        "balances": balances,
        "withdrawals": withdrawals
    }

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/JohnFriedrich/OneDrive/Scripts/Python/RetirementEngine/data/market.csv'

In [5]:
def pad_to_length(stream, target_len):
    if len(stream) < target_len:
        stream += [0.0] * (target_len - len(stream))
    return stream

In [6]:
target_len = 5840  # confirmed index length

for strategy in strategies:
    results[strategy]["withdrawals"] = pad_to_length(results[strategy]["withdrawals"], target_len)

KeyError: 'Fixed'

In [7]:
import pandas as pd

years = list(range(1, len(results[strategies[0]]["balances"]) + 1))
df = pd.DataFrame(index=years)

for strategy in strategies:
    df[f"{strategy} Balance"] = results[strategy]["balances"]
    df[f"{strategy} Withdrawal"] = results[strategy]["withdrawals"]

df.index.name = "Year"
df.style.format("${:,.2f}")

KeyError: 'Fixed'

In [8]:
years = list(range(1, len(results[strategies[0]]["balances"]) + 1))
print("Index length:", len(years))  # Should be 5840

KeyError: 'Fixed'

In [9]:
for strategy in strategies:
    b_len = len(results[strategy]["balances"])
    w_len = len(results[strategy]["withdrawals"])
    print(f"{strategy}: balances={b_len}, withdrawals={w_len}")

KeyError: 'Fixed'

In [9]:
target_len = 5840  # confirmed index length

for strategy in strategies:
    results[strategy]["withdrawals"] = pad_to_length(results[strategy]["withdrawals"], target_len)

KeyError: 'Fixed'

In [10]:
def aggregate_to_yearly(stream, days_per_year=365):
    return [
        sum(stream[i * days_per_year : (i + 1) * days_per_year])
        for i in range(len(stream) // days_per_year)
    ]

In [11]:
yearly_results = {}

for strategy in strategies:
    daily_balances = results[strategy]["balances"]
    daily_withdrawals = results[strategy]["withdrawals"]

    yearly_balances = [
        daily_balances[(i + 1) * 365 - 1]  # last day of each year
        for i in range(len(daily_balances) // 365)
    ]
    yearly_withdrawals = aggregate_to_yearly(daily_withdrawals)

    yearly_results[strategy] = {
        "balances": yearly_balances[:30],
        "withdrawals": yearly_withdrawals[:30]
    }

KeyError: 'Fixed'

In [12]:
df.head(30)  # Show first 30 rows

NameError: name 'df' is not defined

In [13]:
for strategy in strategies:
    print(f"{strategy} length: {len(results[strategy]['balances'])}")
    

KeyError: 'Fixed'

In [14]:
def aggregate_daily_to_annual(results, strategies, years=30, days_per_year=365):
    annual = {}

    for strategy in strategies:
        daily_balances = results[strategy]["balances"]
        daily_withdrawals = results[strategy]["withdrawals"]

        # Ensure we don't exceed available data
        max_years = min(years, len(daily_balances) // days_per_year)

        # Year-end balances: last day of each year
        yearly_balances = [
            daily_balances[(i + 1) * days_per_year - 1]
            for i in range(max_years)
        ]

        # Annual withdrawals: sum of each year's daily values
        yearly_withdrawals = [
            sum(daily_withdrawals[i * days_per_year : (i + 1) * days_per_year])
            for i in range(max_years)
        ]

        annual[strategy] = {
            "balances": yearly_balances,
            "withdrawals": yearly_withdrawals
        }

    return annual

In [15]:
yearly_results = aggregate_daily_to_annual(results, strategies)

KeyError: 'Fixed'

In [10]:
years = list(range(1, len(yearly_results[strategies[0]]["balances"]) + 1))
df = pd.DataFrame(index=years)
df.index.name = "Year"

for strategy in strategies:
    df[f"{strategy} Balance"] = yearly_results[strategy]["balances"]
    df[f"{strategy} Withdrawal"] = yearly_results[strategy]["withdrawals"]

df  # Display the table

NameError: name 'yearly_results' is not defined

In [12]:
# Parameters
withdrawal_rate = 0.05
sp500_weight = 0.60
etf_source = "market.csv"
initial_balance = 1_000_000
strategies = ["Fixed", "Guardrails", "Pause After Loss", "Dynamic"]

# Run simulation for each strategy
results = {}

for strategy in strategies:
    balances, withdrawals = run_simulation(
        withdrawal_rate=withdrawal_rate,
        sp500_weight=sp500_weight,
        etf_source=etf_source,
        strategy=strategy,
        initial_balance=initial_balance
    )
    results[strategy] = {
        "balances": balances,
        "withdrawals": withdrawals
    }

# Aggregate to annual
def aggregate_daily_to_annual(results, strategies, years=30, days_per_year=365):
    annual = {}
    for strategy in strategies:
        daily_balances = results[strategy]["balances"]
        daily_withdrawals = results[strategy]["withdrawals"]

        max_years = min(years, len(daily_balances) // days_per_year)

        yearly_balances = [
            daily_balances[(i + 1) * days_per_year - 1]
            for i in range(max_years)
        ]
        yearly_withdrawals = daily_withdrawals[:max_years]  # already annual

        annual[strategy] = {
            "balances": yearly_balances,
            "withdrawals": yearly_withdrawals
        }

    return annual

yearly_results = aggregate_daily_to_annual(results, strategies)

# Build DataFrame
import pandas as pd
years = list(range(1, len(yearly_results[strategies[0]]["balances"]) + 1))
df = pd.DataFrame(index=years)
df.index.name = "Year"

for strategy in strategies:
    df[f"{strategy} Balance"] = yearly_results[strategy]["balances"]
    df[f"{strategy} Withdrawal"] = yearly_results[strategy]["withdrawals"]

# Display
df.style.format("${:,.2f}")

TypeError: unsupported operand type(s) for +: 'int' and 'list'